In [1]:
from azureml.core import Workspace, Model, Dataset, Datastore, Experiment, Environment, ScriptRunConfig, RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.core.environment import CondaDependencies
import joblib
import os
import pandas as pd
from datetime import datetime

In [2]:
### connecting to ML workspace
### --------------------

subscription_id = '8c386bb9-fbb5-45dd-a9cd-2ca847235881'
resource_group = 'rg-aml-ws-ga'
workspace_name = 'aml-ws-ga'

ws = Workspace(subscription_id, resource_group, workspace_name)

In [3]:
### create compute target
### --------------------

cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                           min_nodes = 0, max_nodes=2)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
### create directory for script
### --------------------

os.makedirs('./scripts', exist_ok= True)

In [11]:
%%writefile scripts/batch_score.py

### batch_score.py
### --------------------

from azureml.core import Workspace, Model, Dataset, Datastore
from azureml.core.authentication import ServicePrincipalAuthentication #
import joblib
import pandas as pd
from datetime import datetime

subscription_id = '8c386bb9-fbb5-45dd-a9cd-2ca847235881'
resource_group = 'rg-aml-ws-ga'
workspace_name = 'aml-ws-ga'

####
 
#svc_pr_password = run.get_secret('mlhero-aml-ws-ga')
svc_pr_password = 'GdCl.3VDLIc.O_0DQxMBxKRaw9k8D~hMps'

svc_pr = ServicePrincipalAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47", service_principal_id="c8ef5ed8-7a7d-4831-984e-571b9c646603", service_principal_password=svc_pr_password)

###

ws = Workspace(subscription_id, resource_group, workspace_name, auth = svc_pr)

### Load data for scoring
df = Dataset.get_by_name(ws, 'german_credit_hsg').to_pandas_dataframe()
df.drop('Sno', axis = 1, inplace = True)
new_data = df[9:16]

### Load model for scoring
model = Model(workspace = ws, name='german-credit-hsg')
model.download()
loaded_model = joblib.load('model.pkl')

### Score new data
results = loaded_model.predict(new_data)
new_data['prediction'] = results

### write output csv
now = datetime.now()
now = now.strftime("%Y_%m_%d__%H_%M_%S")
filename = now + '.csv'
new_data.to_csv(filename)

### upload csv to datastore
ds = Datastore.get_default(ws)
ds.upload_files([filename], target_path = './predictions')


Overwriting scripts/batch_score.py


In [12]:
### defining run environment
### --------------------

myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
conda_dep.add_pip_package("joblib")
conda_dep.add_pip_package("pandas")
conda_dep.add_pip_package("sklearn")
conda_dep.add_pip_package("azureml-sdk")

myenv.python.conda_dependencies=conda_dep

In [14]:
### creating run config
### --------------------

rc = RunConfiguration(script= './scripts/batch_score.py', conda_dependencies= conda_dep)

In [15]:
### create pipeline step
### --------------------

score_step = PythonScriptStep(name = 'Score step', script_name = 'batch_score.py', source_directory = './scripts', compute_target = compute_target, runconfig = rc, allow_reuse = False)


In [16]:
### create & validate pipeline
### --------------------

steps = [score_step]

pipeline = Pipeline(workspace=ws, steps=steps)
pipeline.validate()

Step Score step is ready to be created [6a82a34d]


[]

In [17]:
experiment = Experiment(ws, 'Batch-Scoring-Remote2')

run = experiment.submit(pipeline)

Created step Score step [6a82a34d][56f60598-da68-46e7-acb0-32fb449fb55c], (This step will run and generate new outputs)
Submitted PipelineRun 53131a41-4b21-418c-af10-506926a31cd4
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/Batch-Scoring-Remote2/runs/53131a41-4b21-418c-af10-506926a31cd4?wsid=/subscriptions/8c386bb9-fbb5-45dd-a9cd-2ca847235881/resourcegroups/rg-aml-ws-ga/workspaces/aml-ws-ga


In [ ]:
### publish pipeline
### --------------------

#pipeline.publish(name='pred-ops-concept-test')